<a href="https://colab.research.google.com/github/RajkumarBahalia/CountryList/blob/master/NiftyMarketDepth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io
import warnings
import pandas as pd
from google.colab import files

# Optional: silence the "Could not infer format" warning from pd.to_datetime
warnings.filterwarnings(
    "ignore",
    message="Could not infer format, so each element will be parsed individually",
    category=UserWarning
)

# If you know your date format, set it here (examples):
# DATE_FORMAT = "%Y-%m-%d"    # e.g. 2025-11-23
# DATE_FORMAT = "%d-%m-%Y"    # e.g. 23-11-2025
DATE_FORMAT = None  # leave as None if you're not sure


# ========== 1. Helper functions ==========

def compute_daily_changes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute daily % change, Up% and Down% compared to previous day Close.
    Assumes df index is Date and 'Close' column exists.
    """
    df = df.copy()

    df["Prev_Close"] = df["Close"].shift(1)

    df["Daily_Pct_Change"] = (
        (df["Close"] - df["Prev_Close"]) / df["Prev_Close"] * 100
    ).round(2)

    # Up and Down split
    df["Daily_Up_by_%"] = df["Daily_Pct_Change"].where(df["Daily_Pct_Change"] > 0, 0)
    df["Daily_Down_by_%"] = (-df["Daily_Pct_Change"]).where(df["Daily_Pct_Change"] < 0, 0)

    # Replace NaN in first row with 0
    cols_to_clean = ["Prev_Close", "Daily_Pct_Change", "Daily_Up_by_%", "Daily_Down_by_%"]
    df[cols_to_clean] = df[cols_to_clean].fillna(0)

    return df


def compute_weekly_changes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Resample to weekly OHLC (Friday close) and compute
    % up/down from previous week's close.
    Assumes df index is Date and has OHLC columns.
    """
    # Resample to weekly (Friday) OHLC
    weekly = df.resample("W-FRI").agg(
        Open=("Open", "first"),
        High=("High", "max"),
        Low=("Low", "min"),
        Close=("Close", "last"),
    ).dropna()

    # Previous week close
    weekly["Prev_Week_Close"] = weekly["Close"].shift(1)

    # % change from previous week's close
    weekly["Weekly_Pct_Change"] = (
        (weekly["Close"] - weekly["Prev_Week_Close"])
        / weekly["Prev_Week_Close"].where(weekly["Prev_Week_Close"] != 0)
        * 100
    )

    # First week will have NaN -> set to 0 and round
    weekly["Weekly_Pct_Change"] = weekly["Weekly_Pct_Change"].fillna(0).round(2)

    # Up / Down split
    weekly["Weekly_Up_by_%"] = weekly["Weekly_Pct_Change"].clip(lower=0)
    weekly["Weekly_Down_by_%"] = (-weekly["Weekly_Pct_Change"]).clip(lower=0)

    return weekly


# ========== 2. Upload & read CSV ==========

print("Please select your OHLC CSV file:")
uploaded = files.upload()

filename, filedata = next(iter(uploaded.items()))
df = pd.read_csv(io.StringIO(filedata.decode("utf-8")))

# Convert 'Date' to datetime
if DATE_FORMAT is not None:
    # Use explicit format (recommended if you know it)
    df["Date"] = pd.to_datetime(df["Date"], format=DATE_FORMAT, errors="coerce")
else:
    # Generic parser (may show a warning, which we silenced above)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Drop rows where Date could not be parsed
df = df.dropna(subset=["Date"])

# Sort by date just in case
df = df.sort_values("Date")

# Set Date as index
df = df.set_index("Date")


# ========== 3. Daily and Weekly calculations ==========

# Daily stats
daily_df = compute_daily_changes(df)

# Weekly stats
weekly_df = compute_weekly_changes(df)

# ---------- Make clean output DataFrames ----------

# DAILY: ensure we have a 'Date' column coming from the index, without clashes
daily_out = daily_df.copy()
if "Date" not in daily_out.columns:
    daily_out["Date"] = daily_out.index
# Drop old index, keep only columns
daily_out = daily_out.reset_index(drop=True)

# WEEKLY: create explicit 'Week_End_Date' column from index (no reset_index() magic)
weekly_out = weekly_df.copy()
weekly_out["Week_End_Date"] = weekly_out.index
weekly_out = weekly_out.reset_index(drop=True)


# ========== 4. Save & (optionally) download ==========

daily_path = "nifty_ohlc_daily_with_pct_change.csv"
weekly_path = "nifty_ohlc_weekly_with_pct_change.csv"

daily_out.to_csv(daily_path, index=False)
weekly_out.to_csv(weekly_path, index=False)

print("Done!")
print(f"Daily file saved as:  {daily_path}")
print(f"Weekly file saved as: {weekly_path}")

print("\nLast few weekly rows:")
print(weekly_out.tail())

# To trigger download in Colab, uncomment these:
# files.download(daily_path)
# files.download(weekly_path)
